In [1]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os


In [2]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [3]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [4]:
import functions
import transformers
import importlib

importlib.reload(functions)
importlib.reload(transformers)
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer





from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)






<module 'transformers.one_hot_encoder_transformer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\transformers\\one_hot_encoder_transformer.py'>

In [5]:
names = ['BetterBot', 'STEEBot', 'HastyBot']
bots = BotExtarctor(names)(train)
is_player = ~IsBotExtarctor(names)(train)


In [6]:
test = train
transformer = SelectTransformer(is_player)
transformer.transform(test)


,nickname,score,rating
game_id,,,
1,stevy,429,1500
3,davidavid,440,1811
4,Inandoutworker,119,1473
5,stevy,325,1500
6,HivinD,378,2029
...,...,...,...
72767,HAYDEN,340,1395
72770,samsiah06,97,1332
72771,BB-8,390,1500


In [7]:
train_pipe = (Pipeline([('bot_names', ColumnsSetterTransformer({'bot_name': BotExtarctor(names),})),
                   ('players_selection', SelectTransformer(is_player)),
                   ('set_bot_training', ColumnsSetterTransformer({'bot_rating': train[~is_player]['rating']})),
                   ('name_dropper', NameDropperTransformer(['nickname']))
                   ]))

games_pipe = (Pipeline([('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                        ('drop_first', NameDropperTransformer('first'))
                       ]))

new_train = train_pipe.transform(train)
new_games = games_pipe.transform(games).merge(new_train, left_index=True, right_index=True)

In [8]:
# new_games = games.merge(new_train, left_index=True, right_index=True)
# new_games

In [9]:
# is_player = ~IsBotExtarctor(names)(train)
# bots = BotExtarctor(names)(train)
# train_no_bots = train[is_player]
# train_no_bots['bot_name'] = bots
# train_no_bots['bot_rating'] = train[~is_player]['rating']
# train_no_bots = train_no_bots.drop(columns=['nickname'])
# games_with_rating = games.merge(train_no_bots, left_index=True, right_index=True)
# train_no_bots


In [10]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [11]:
trans = SelectRowsTransformer(train.index)
train_turns = trans.transform(turns)

In [12]:
turns_pipe = Pipeline([('is_player_extractor', ColumnsSetterTransformer({'is_player_turn': IsBotExtarctor(names, neg=True)})),
                       ('word_info_mappers', MapSetTransformer({'move_len': (lambda x: len(x) if type(x) == str else np.NAN, 'move'),
                                                                'reused_num': (lambda x: x.count('.') if type(x) == str else np.NAN, 'move'),
                                                                'jokers_num': (lambda x: sum(1 for c in x if c.islower()) if (type(x) == str) and (x not in ['(challenge)', '(time)']) else np.NAN, 'move')
                                                                })),
                       ('to_series', SeriesFromGroupTransformer('game_id', 'turn_number'))
                       ])

new_turns = turns_pipe.transform(turns)
new_turns

1                      nickname     rack location      ...
2                      nickname     rack location      ...
3                      nickname     rack location      ...
4                           nickname     rack location ...
5                    nickname     rack location       m...
                               ...                        
72769                 nickname     rack location      m...
72770                  nickname     rack location      ...
72771                 nickname     rack location     mo...
72772                  nickname     rack location      ...
72773                 nickname     rack location     mo...
Length: 72773, dtype: object

In [13]:
more_games_pipe = Pipeline([('assign_turns', ColumnsSetterTransformer({'turns': new_turns}))])

new_games = more_games_pipe.transform(new_games)
new_games.head()

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating,turns
game_id,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637,nickname rack location ...
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071,nickname rack location ...
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936,nickname rack location ...
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844,nickname rack location m...
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143,nickname rack location mo...


In [14]:
new_games['turns'][4]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,BetterBot,ADEMSTX,8D,MAXED,36,36,Play,False,5.0,0.0,0.0
2,Inandoutworker,DISTVWY,9E,WIT,24,24,Play,True,3.0,0.0,0.0
3,BetterBot,DEJRSTU,7G,JET,40,76,Play,False,3.0,0.0,0.0
4,Inandoutworker,ADLOSVY,J7,SLAY,26,50,Play,True,4.0,0.0,0.0
5,BetterBot,DERRRSU,E4,REDR..,20,96,Play,False,6.0,2.0,0.0
6,Inandoutworker,BBDGOUV,NaN,-BBGOUV,0,50,Exchange,True,7.0,0.0,0.0
7,BetterBot,EFIMRSU,D1,FUME,30,126,Play,False,4.0,0.0,0.0
8,Inandoutworker,DEEEGOR,1A,REE.,21,71,Play,True,4.0,1.0,0.0
9,BetterBot,EINNORS,E4,......N,11,137,Play,False,7.0,6.0,0.0


In [15]:
moments_pipe = Pipeline([('points_moments', MapSetTransformer({'p_points_1_moment': (lambda turns: (turns[turns['is_player_turn']]['points']).mean(), 'turns'),
                                                            'p_points_std': (lambda turns: turns[turns['is_player_turn']]['points'].std(), 'turns'),
                                                            'b_points_1_moment': (lambda turns: (turns[~turns['is_player_turn']]['points']).mean(), 'turns'),
                                                            'b_points_std': (lambda turns: turns[~turns['is_player_turn']]['points'].std(), 'turns'),
                                                            'a_points_1_moment': (lambda turns: (turns['points']).mean(), 'turns'),
                                                            'a_points_std': (lambda turns: turns['points'].std(), 'turns'),
                                                            },
))])
res = moments_pipe.transform(new_games)


              

In [16]:
res['turns'][61248]

,nickname,rack,location,move,points,score,turn_type,is_player_turn,move_len,reused_num,jokers_num
turn_number,,,,,,,,,,,
1,CodeTitan,DLNPSUY,8H,PUS,10,10,Play,True,3.0,0.0,0.0
2,HastyBot,EEIIORT,G8,OI,9,9,Play,False,2.0,0.0,0.0
3,CodeTitan,ADDILNY,8F,Y....,10,20,Play,True,5.0,4.0,0.0
4,CodeTitan,ADDILNY,NaN,--,-10,10,Exchange,True,2.0,0.0,0.0
5,HastyBot,CEEIIRT,J5,ICE.,12,21,Play,False,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
63,CodeTitan,ADDILNY,J13,.Y,13,23,Play,True,2.0,1.0,0.0
64,CodeTitan,ADDILNY,NaN,--,-13,10,Exchange,True,2.0,0.0,0.0
65,HastyBot,GT,J12,T.G,8,781,Play,False,3.0,1.0,0.0


In [17]:
turns[turns['move'].isna()]

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
377,27,NurdiniDelisha,ADFNTTU,O8,NaN,16,249,NaN
409,22,akiff_faleel,AANU,14J,NaN,12,350,NaN
420,23,Rick,AGINU,M12,NaN,19,409,NaN
536,13,HastyBot,AAEILNN,F5,NaN,14,316,NaN
630,2,220674_ZAVEPHUA,AAEINNR,7H,NaN,11,11,NaN
...,...,...,...,...,...,...,...,...
72252,10,akiff_faleel,CGLLNUX,D10,NaN,12,238,Play
72395,18,gppa,AEINRSU,A14,NaN,35,272,NaN
72467,7,Saryk,?AALNOU,9F,NaN,11,74,NaN


In [18]:
pd.get_dummies(games['lexicon'], prefix='lexicon', dummy_na=True).sum()


lexicon_CSW21     44594
lexicon_ECWL       6925
lexicon_NSWL20        4
lexicon_NWL20     21250
lexicon_nan           0
dtype: int64

In [19]:
trans = OneHotEncoderTransformer(['lexicon', 'rating_mode'])
trans.transform(games)

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,lexicon_nan,rating_mode_CASUAL,rating_mode_RATED,rating_mode_nan
game_id,,,,,,,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,0,0,0,1,0,1,0,0
2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418,1,0,0,0,0,0,1,0
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,1,0,0,0,0,0,1,0
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1,0,0,0,0,1,0,0
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72769,browndav,blitz,STANDARD,0,2022-09-07 13:24:13,CSW21,180,0,RATED,1,218.043751,1,0,0,0,0,0,1,0
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,1,0,0,0,0,0,1,0
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,0,1,0,0,0,1,0,0


In [20]:
games.isna().sum()

first                    0
time_control_name        0
game_end_reason          0
winner                   0
created_at               0
lexicon                  0
initial_time_seconds     0
increment_seconds        0
rating_mode              0
max_overtime_minutes     0
game_duration_seconds    0
dtype: int64

In [21]:
# need to start cv and gridsearch

In [22]:
new_games

,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,is_bot_first,score,rating,bot_name,bot_rating,turns
game_id,,,,,,,,,,,,,,,,
1,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,True,429,1500,BetterBot,1637,nickname rack location ...
3,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,True,440,1811,BetterBot,2071,nickname rack location ...
4,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,True,119,1473,BetterBot,1936,nickname rack location ...
5,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,True,325,1500,STEEBot,1844,nickname rack location m...
6,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,False,378,2029,STEEBot,2143,nickname rack location mo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,True,340,1395,BetterBot,1952,nickname rack location ...
72770,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,True,97,1332,HastyBot,2237,nickname rack location ...
72771,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,True,390,1500,HastyBot,1614,nickname rack location mo...


In [25]:
y = new_games['rating']
hot_drop_pipe = Pipeline([('hot', OneHotEncoderTransformer(['time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),
                        ('drops', NameDropperTransformer(['created_at', 'rating', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name', 'turns']))])

x = hot_drop_pipe.transform(new_games)
x

,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,is_bot_first,score,bot_rating,time_control_name_blitz,time_control_name_rapid,...,lexicon_NSWL20,lexicon_NWL20,lexicon_nan,rating_mode_CASUAL,rating_mode_RATED,rating_mode_nan,bot_name_BetterBot,bot_name_HastyBot,bot_name_STEEBot,bot_name_nan
game_id,,,,,,,,,,,,,,,,,,,,,
1,1,1200,0,1,674.844274,True,429,1637,0,0,...,0,1,0,1,0,0,1,0,0,0
3,1,900,0,5,492.268262,True,440,2071,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,3600,0,1,350.861141,True,119,1936,0,0,...,0,0,0,1,0,0,1,0,0,0
5,0,1200,0,1,642.688722,True,325,1844,0,0,...,0,1,0,1,0,0,0,0,1,0
6,0,900,0,1,426.950541,False,378,2143,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0,3600,0,1,3669.988413,True,340,1952,0,0,...,0,0,0,1,0,0,1,0,0,0
72770,0,3600,0,1,3662.659287,True,97,2237,0,0,...,0,0,0,0,1,0,0,1,0,0
72771,0,900,0,10,236.133634,True,390,1614,0,0,...,0,0,0,1,0,0,0,1,0,0


In [26]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

RandomForestRegressor(max_depth=2, random_state=0)

In [27]:
from sklearn.metrics import mean_squared_error
y_pred = regr.predict(x)
np.sqrt(mean_squared_error(y, y_pred))

160.6309598124796

In [ ]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [28]:
dict = {'games': games, 'turns': turns, 'train': train}

In [49]:
testpipe = Pipeline([('test_turns_remove', SelectRowsTransformer(train.index, target='turns'))])



res = testpipe.transform(dict)
res['turns']

KeyboardInterrupt: 

In [ ]:
temp = dict['turns']
temp.loc[train.index, :]

In [55]:
temp = turns
temp.loc[train.index, :]

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [ ]:
check = SelectRowsTransformer(train.index, target='turns').transform(dict)

In [48]:
check['turns'].loc[2, :]

KeyError: 2